# LabCup Data Cleaning

In [1]:
import pandas as pd
from pymongo import MongoClient
import socket
import sys

# Load the Excel file
df = pd.read_excel("/home/ian/datalab-deployment/datalab/Chemicals_all_09_06_2025.xlsx")

## Data Cleaning

In [2]:
# Display initial dataframe info
print("Initial dataset shape:", df.shape)
print("\nColumn names:")
print(df.columns.tolist())
print("\nFirst few rows:")
df.head()

Initial dataset shape: (34809, 106)

Column names:
['Barcode', 'Brand', 'Product number', 'Product name', 'CAS', 'Lot Number', 'CID', 'InChI', 'Iupac Name', 'Quantity', 'Net Quantity', 'Quantity unit', 'Current quantity (Net)', 'Normalized quantity', 'Normalized current quantity', 'Normalized unit', 'Group', 'Responsible person', 'Site', 'Building', 'Laboratory', 'Location path', 'Sublocation', 'Proposed location', 'Date in', 'Expire', 'UN number', 'Packaging group', 'Hazard Signal Word', 'Hazard Label(s)', 'NFPA Label', 'German Water Hazard Class (WGK)', 'Lagerklasse (TRGS 510)', 'Transport hazard class 1', 'Transport hazard class 2', 'Transport hazard class 3', 'Hazard Classification', 'Hazard code(s)', 'Region specific hazard code(s)', 'IFC classification', 'Precautionary statement(s)', 'Purity (%)', 'State', 'Storage temperature', 'Tags', 'Approved by', 'Container type', 'Note (Item)', 'Note (Product)', 'Personal protective equipment (normal use)', 'Personal protective equipment (e

,Barcode,Brand,Product number,Product name,CAS,Lot Number,CID,InChI,Iupac Name,Quantity,...,Component - concentration.5,Component - CAS.6,Component - name.6,Component - concentration.6,Component - CAS.7,Component - name.7,Component - concentration.7,Component - CAS.8,Component - name.8,Component - concentration.8
0,CH009014,unknown,unknown,Zinc fluoride 97%,7783-49-5,NaN,24551.0,InChI=1S/2FH.Zn/h2*1H;/q;;+2/p-2,NaN,250 G,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CH009470,unknown,unknown,Calcium oxide 99.995%,1305-78-8,NaN,14778.0,InChI=1S/Ca.O,NaN,499.9 G,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CH009053,unknown,unknown,Copper(II) sulfide,1317-40-4,NaN,14831.0,InChI=1S/Cu.S,NaN,10 G,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CH009071,unknown,unknown,Copper(II) oxide 97%,1317-38-0,NaN,14829.0,InChI=1S/Cu.O,NaN,100 G,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CH009017,unknown,unknown,Zinc oxide 99+%,1314-13-2,NaN,14806.0,InChI=1S/O.Zn,NaN,247.956 G,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:

# get rid of all the columns that are completely empty
df = df.dropna(axis=1, how='all')
# print("After dropping empty columns:", df.shape)
# Rename the columns to match DataLab names, this needs to be the same as the names in the DataLab json or else they wont be updated to the corresponding fields
df = df.rename(columns={
    "Product name": "name",
    "Brand": "supplier",
    "Molecular formula": "chemform",
    "Molecular weight (g/mol)" : "molar_mass",
    'Hazard code(s)': 'GHS_codes', 
})
# print("Columns after renaming:", df.columns.tolist())
# G should be g in 'Quantity'
df['Quantity'] = df['Quantity'].str.lower()

# location should be the combination of Building-Laboratory-Location Path-Sublocation
df['location'] = df['Building'] + '-' + df['Laboratory'] + '-' + df['Location path'] + '-' + df['Sublocation']
df

,Barcode,supplier,Product number,name,CAS,Lot Number,CID,InChI,Iupac Name,Quantity,...,Component - name.4,Component - CAS.5,Component - name.5,Component - CAS.6,Component - name.6,Component - CAS.7,Component - name.7,Component - CAS.8,Component - name.8,location
0,CH009014,unknown,unknown,Zinc fluoride 97%,7783-49-5,NaN,24551.0,InChI=1S/2FH.Zn/h2*1H;/q;;+2/p-2,NaN,250 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CH009470,unknown,unknown,Calcium oxide 99.995%,1305-78-8,NaN,14778.0,InChI=1S/Ca.O,NaN,499.9 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CH009053,unknown,unknown,Copper(II) sulfide,1317-40-4,NaN,14831.0,InChI=1S/Cu.S,NaN,10 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CH009071,unknown,unknown,Copper(II) oxide 97%,1317-38-0,NaN,14829.0,InChI=1S/Cu.O,NaN,100 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CH009017,unknown,unknown,Zinc oxide 99+%,1314-13-2,NaN,14806.0,InChI=1S/O.Zn,NaN,247.956 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34804,CH043011,Sigma-Aldrich,D194255,"1,3-Dinitrobenzene 97%",99-65-0,NaN,7452.0,InChI=1S/C6H4N2O4/c9-7(10)5-2-1-3-6(4-5)8(11)1...,NaN,25 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34805,CH043012,Sigma-Aldrich,516988,Cesium hydroxide monohydrate 99.95% trace meta...,35103-79-8,NaN,23679066.0,InChI=1S/Cs.2H2O/h;2*1H2/q+1;;/p-1,NaN,4.5 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34806,CH085636,Fluorochem,11781,1H-Imidazole-2-carbaldehyde,10111-08-7,NaN,24955.0,"InChI=1S/C4H4N2O/c7-3-4-5-1-2-6-4/h1-3H,(H,5,6)",NaN,25 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34807,CH028425,unknown,unknown,3-(Dimethylamino)-1-propylamine 98%,109-55-7,NaN,7993.0,"InChI=1S/C5H14N2/c1-7(2)5-3-4-6/h3-6H2,1-2H3",NaN,250 ml,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# empty CAS
empty_cas = df[df['CAS'].isna()]
print("Number of rows with empty CAS:", len(empty_cas))
# empty InChI
empty_inchi = df[df['InChI'].isna()]
print("Number of rows with empty InChI:", len(empty_inchi))

# find out how many rows have both empty CAS and InChI
empty_cas_inchi = df[(df['CAS'].isna()) & (df['InChI'].isna())]
print("Number of rows with both empty CAS and InChI:", len(empty_cas_inchi))
print("Rows with both empty CAS and InChI:")
print(empty_cas_inchi)

Number of rows with empty CAS: 2982
Number of rows with empty InChI: 7661
Number of rows with both empty CAS and InChI: 2976
Rows with both empty CAS and InChI:
        Barcode                  supplier Product number  \
21     CH100902             Sigma-Aldrich   8.20543.1000   
40     CH091547                    Restek         579540   
41     CH091550                    Restek         576368   
43     CH091548             Sigma-Aldrich        44078-U   
46     CH082846      SIGMA-ALDRICH CO LTD      289191-5g   
...         ...                       ...            ...   
29800  CH110150                Fluorochem     M03403-10g   
29806  CH095659                Fluorochem     F601412-1g   
29807  CH110192                Fluorochem    F040536-25g   
29808  CH110172                Fluorochem    F237023-10g   
29860  CH095669  Thermo Fisher Scientific          14486   

                                                    name  CAS Lot Number  CID  \
21                                   

## InChI to Chemical Formula

In [5]:
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors

def inchi_to_formula(inchi_str):
    """
    Convert InChI to molecular formula.
    """
    mol = Chem.MolFromInchi(inchi_str)
    if mol is not None:
        formula = rdMolDescriptors.CalcMolFormula(mol)
        return formula
    else:
        print(f"Invalid InChI: {inchi_str}")
        return None

In [6]:
# empty Chemical formula column but has InChI
print('Number of rows with empty Chemical formula before conversion:', len(df[df['chemform'].isna()]))
empty_formula = df[(df['chemform'].isna()) & (df['InChI'].notna())]
print("Number of rows with empty Chemical formula but valid InChI:", len(empty_formula))
# Convert InChI to Chemical formula for these rows
for index, row in empty_formula.iterrows():
    inchi_str = row['InChI']
    formula = inchi_to_formula(inchi_str)
    if formula:
        df.at[index, 'chemform'] = formula
# Check if there are still any empty Chemical formula entries
empty_formula_after = df[df['chemform'].isna()]
print("Number of rows with empty Chemical formula after conversion:", len(empty_formula_after))

Number of rows with empty Chemical formula before conversion: 7328
Number of rows with empty Chemical formula but valid InChI: 23
Number of rows with empty Chemical formula after conversion: 7305


In [7]:
# save this to a new excel file
df.to_excel("/home/ian/datalab-deployment/datalab/Chemicals_all_09_06_2025_updated.xlsx", index=False)

## CAS to Chemical Formula using Online Data (Currently redundant)

Function to extract wikipedia CAS list

In [ ]:
# extract chemical species and CAS numbers from https://en.wikipedia.org/wiki/List_of_CAS_numbers_by_chemical_compound
# store the results as a Pandas DataFrame
# use the CAS numbers as the index
# use beautifulsoup to extract the data from the webpage
def extract_wikipedia_CAS_list():
    """
    Extracts chemical species and CAS numbers from https://en.wikipedia.org/wiki/List_of_CAS_numbers_by_chemical_compound.
    Returns the results as a Pandas DataFrame.
    """

    import requests
    from bs4 import BeautifulSoup

    # download the webpage
    response = requests.get(
        "https://en.wikipedia.org/wiki/List_of_CAS_numbers_by_chemical_compound"
    )

    # parse the html
    soup = BeautifulSoup(response.text, "html.parser")

    # find the table containing the data
    all_tables = soup.find_all("table")

    # extract the data from the table
    data = []

    for table in all_tables:
        for row in table.find_all("tr"):
            row_data = []
            for cell in row.find_all(["th", "td"]):
                row_data.append(cell.text.strip())
            if "CAS number" not in row_data or len(data) == 0:
                data.append(row_data)

    # convert the data to a Pandas DataFrame
    df = pd.DataFrame(data[1:], columns=data[0])

    # set the CAS number as the index
    df.set_index("CAS number", inplace=True)

    # Replace \u2022 (middle dot) with a .. in the chemical formula column to parse hydrates
    df["Chemical formula"] = df["Chemical formula"].str.replace("\u2022", "..")

    return df

In [ ]:
# df that can be used to look up CAS numbers and chemical formulas
cas_to_formula_df = extract_wikipedia_CAS_list()
# use this df to map CAS numbers to chemical formulas in the main df
empty_formula = df[(df['Chemical formula'].isna()) & (df['CAS'].notna())]
print("Number of rows with empty Chemical formula but valid CAS:", len(empty_formula))
for index, row in empty_formula.iterrows():
    cas_number = row['CAS']
    if cas_number in cas_to_formula_df.index:
        formula = cas_to_formula_df.loc[cas_number, 'Chemical formula']
        df.at[index, 'Chemical formula'] = formula
    else:
        print(f"CAS number {cas_number} not found in Wikipedia data.")


Number of rows with empty Chemical formula but valid CAS: 4345
CAS number 7783-50-8 not found in Wikipedia data.
CAS number 7789-28-8 not found in Wikipedia data.
CAS number 13470-08-1 not found in Wikipedia data.
CAS number 107091-89-4 not found in Wikipedia data.
CAS number 7783-71-3 not found in Wikipedia data.
CAS number 223463-14-7 not found in Wikipedia data.
CAS number 925-93-9 not found in Wikipedia data.
CAS number 358-23-6 not found in Wikipedia data.
CAS number 358-23-6 not found in Wikipedia data.
CAS number 583-58-4 not found in Wikipedia data.
CAS number 58556-75-5 not found in Wikipedia data.
CAS number 14808-60-7 not found in Wikipedia data.
CAS number 627-37-2 not found in Wikipedia data.
CAS number 36953-42-1 not found in Wikipedia data.
CAS number 153086-78-3 not found in Wikipedia data.
CAS number 79060-88-1 not found in Wikipedia data.
CAS number 329214-79-1 not found in Wikipedia data.
CAS number 72287-26-4 not found in Wikipedia data.
CAS number 1493-13-6 not fou

In [ ]:
# Check if there are still any empty Chemical formula entries
empty_formula_after = df[df['Chemical formula'].isna()]
print("Number of rows with empty Chemical formula after conversion:", len(empty_formula_after))

Number of rows with empty Chemical formula after conversion: 7304


In [ ]:
# number of rows with name but not chemical formula
empty_formula = df[(df['Chemical formula'].isna()) & (df['Name'].notna())]
print("Number of rows with empty Chemical formula but valid Name:", len(empty_formula))

Number of rows with empty Chemical formula but valid Name: 7304
